In [ ]:
from Engine.Tools import phiFunction, relativeAngle
from itertools import permutations
import pandas as pd
import numpy as np
import random

In [ ]:
# Simulation setup
random.seed(123)
N: int = 1000                                                           # Simulation size
Lag: int = 1                                                            # Lag value in the model

In [8]:
# Geographical setup
GridSize: list = [x for x in range(-1, 2)]                                           # Size of the geographical grid for the simulation
Distance: float = 20.0                                                               # in Km
Location: dict = {i: x for i, x in enumerate(list(permutations(GridSize, r=2)))}     # Dictionary of coordinates for each integer-valued intersection point in the grid

In [ ]:
# Environmental Setup
windSpeed: np.ndarray = np.random.lognormal(mean=10.0, sigma=20.0, size=[N, 1])                              # Random non-negative wind speed
windDirection: np.ndarray = np.degrees(np.arccos(np.cos(np.linspace(0, 2 * np.pi, N, endpoint=False))))      # Sequential wind direction 

In [ ]:
# Simulating variables
K: int = len(GridSize) ** 2
initialPollution: float = np.random.lognormal(mean=150, sigma=40, size=[1, K])       # Random initial pollution level
Y: np.ndarray = np.zeros([N, K])
Y[0, :] = initialPollution
for i in range(1, N):
    Y[i, :] = phiFunction(
        
    ) @ Y[i-1, :] + np.random.normal(loc=0, scale=1)